In [1]:
import numpy as np
import opendssdirect as dss
import matplotlib.pyplot as plt
import pandas as pdk
import re
import sys
import time
import scipy.linalg as spla
from lib.compute_KCL_matrices import compute_KCL_matrices
from lib.basematricesY import basematrices
from lib.NR3_Y import NR3_timevarying

Vectorized Solver

In [2]:
slackidx = 0
Vslack = np.array([1, np.exp(1j*-120*np.pi/180), np.exp(1j*120*np.pi/180)])

#fn = 'compare_opendss_05node_threephase_unbalanced_oscillation_03.dss'
#fn = '06node_threephase_radial_unbalanced.dss'
#fn = 'IEEE_13Node_Modified_01.dss'
fn = '06node_threephase_meshed_unbalanced.dss'
times = [-1]
der = [0]
capacitance =[0]

X, g_SB, b_SB, G_KVL, b_KVL, H, g, b = basematrices(fn, slackidx, Vslack, None, None)

   
dss.run_command('Redirect ' + fn)
nnode = len(dss.Circuit.AllBusNames())
nline = len(dss.Lines.AllNames())

VNR01 = np.zeros((len(times), 3, nnode), dtype = "complex")
INR01=  np.zeros((len(times), 3, nline), dtype = "complex")
STXNR01 = np.zeros((len(times), 3, nline), dtype = "complex")
SRXNR01 = np.zeros((len(times), 3, nline), dtype = "complex")
iNR01 = np.zeros((len(times), 3, nnode), dtype = "complex")
sNR01 = np.zeros((len(times), 3, nnode), dtype = "complex")


    # Run NR3 with variations in time, DER, and capacitance
# VNR, INR, STXNR, SRXNR, iNR, sNR, itercount = \
VNR = \
    NR3_timevarying(fn, X, g_SB, b_SB, G_KVL, b_KVL, H, g, b, None, None, 0, 0, -1)    
VNR01[0, :, :] = np.reshape(VNR, (3, nnode))
# INR01[i, :, :] = np.reshape(INR, (3, nline))
# STXNR01[i, :, :] = np.reshape(STXNR, (3, nline))
# SRXNR01[i, :, :] = np.reshape(SRXNR, (3, nline))
# iNR01[i, :, :] = np.reshape(iNR, (3, nnode))
# sNR01[i, :, :] = np.reshape(sNR, (3, nnode))


Iteration number 0.000000
Iteration number 1.000000
Iteration number 2.000000
Iteration number 3.000000
Iteration number 4.000000
Iteration number 5.000000
Iteration number 6.000000
Iteration number 7.000000
Iteration number 8.000000
Iteration number 9.000000
Iteration number 10.000000
Iteration number 11.000000
Iteration number 12.000000
Iteration number 13.000000
Iteration number 14.000000
Iteration number 15.000000
Iteration number 16.000000
Iteration number 17.000000
Iteration number 18.000000
Iteration number 19.000000
Iteration number 20.000000
Iteration number 21.000000
Iteration number 22.000000
Iteration number 23.000000
Iteration number 24.000000
Iteration number 25.000000
Iteration number 26.000000
Iteration number 27.000000
Iteration number 28.000000
Iteration number 29.000000
Iteration number 30.000000
Iteration number 31.000000
Iteration number 32.000000
Iteration number 33.000000
Iteration number 34.000000
Iteration number 35.000000
Iteration number 36.000000
Iteration n

OpenDSS Solution

In [3]:
#dss.run_command('Redirect compare_opendss_05node_threephase_unbalanced_oscillation_03.dss')
#dss.run_command('Redirect 06node_threephase_unbalanced.dss') 
#dss.run_command('Redirect IEEE_13Node_Modified_01.dss')
dss.run_command('Redirect 06node_threephase_meshed_unbalanced.dss')

VDSS0 = np.zeros((len(times), 3, nnode), dtype = "complex")
IDSS0 =  np.zeros((len(times), 3, nline), dtype="complex")
SRXDSS0 = np.zeros((len(times), 3, nline), dtype="complex")
STXDSS0 = np.zeros((len(times), 3, nline), dtype="complex")
load_arr = np.zeros((len(times), 3, nnode), dtype='complex')

dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[0])
Vbase = dss.Bus.kVBase() * 1000
Sbase = 1000000.0
Ibase = Sbase/Vbase
Zbase = Vbase/Ibase

# Set slack bus (sourcebus) voltage reference in p.u.
SlackBusVoltage = 1.000
dss.Vsources.PU(SlackBusVoltage)

dss.Solution.Convergence(0.000000000001)

# Solve power flow with OpenDSS file
dss.Solution.Solve()
if not dss.Solution.Converged:
    print('Initial Solution Not Converged. Check Model for Convergence')
else:
    print('Initial Model Converged. Proceeding to Next Step.')
    #Doing this solve command is required for GridPV, that is why the monitors
    #go under a reset process
    dss.Monitors.ResetAll()

    # set solution Params
    #setSolutionParams(dss,'daily',1,1,'off',1000000,30000)
    dss.Solution.Mode(1)
    dss.Solution.Number(1)
    dss.Solution.StepSize(1)
    dss.Solution.ControlMode(-1)
    dss.Solution.MaxControlIterations(1000000)
    dss.Solution.MaxIterations(30000)
    # Easy process to get all names and count of loads, a trick to avoid
    # some more lines of code
    TotalLoads=dss.Loads.Count()
    AllLoadNames=dss.Loads.AllNames()
    print('OpenDSS Model Compliation Done.')
    print('Iterations: ', dss.Solution.Iterations())
    print('Tolerance: ', dss.Solution.Convergence())

print('')


print(dss.Solution.Converged())

# Print number of buses, and bus names
print(len(dss.Circuit.AllBusNames()))
print(dss.Circuit.AllBusNames())

# Print number of loads, and load names
print(len(dss.Loads.AllNames()))
print(dss.Loads.AllNames())

print('')

VDSS = np.zeros((3,len(dss.Circuit.AllBusNames())),dtype='complex')

for k1 in range(len(dss.Circuit.AllBusNames())):

    dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])

#     print(dss.Circuit.AllBusNames()[k1])
#     print(dss.Bus.Nodes())

#     print('puVOTLAGES - LN CARTESIAN')
#     print(dss.Bus.PuVoltage())

    ph = np.asarray(dss.Bus.Nodes(),dtype='int')-1

    Vtemp = np.asarray(dss.Bus.PuVoltage())

    Vtemp = Vtemp[0:nnode*2-1:2] + 1j*Vtemp[1:nnode*2:2]


#     print(np.asarray(dss.Bus.Nodes(),'int'))

    VDSS[ph,k1] = Vtemp    

#     VDSS[np.asarray(dss.Bus.Nodes(),'int'),k1] = np.array(dss.Bus.PuVoltage()[0:5:2] + 1j*dss.Bus.PuVoltage()[1:6:2])


#     VDSS[dss.Bus.Nodes()-1,k1] = dss.Bus.PuVoltage()[0:2:5]
#     for k2 in range(len(dss.Bus.Nodes())):
#         VDSS[int(dss.Bus.Nodes()[k2])-1,k1] = dss.Bus.PuVoltage()[2*k2] + 1j*dss.Bus.PuVoltage()[2*k2+1]

    print('VDSS\n', np.round(VDSS,decimals=6))
    VDSS0[0, :, :] = VDSS

#     IDSS = np.zeros((3,len(dss.Lines.AllNames())),dtype='complex')

#     for k1 in range(len(dss.Lines.AllNames())):
#         dss.Lines.Name(dss.Lines.AllNames()[k1])
#     #     print(dss.Lines.AllNames()[k1])
#         ph = np.asarray(dss.CktElement.BusNames()[0].split('.')[1:], dtype='int')-1
#         Imn = np.asarray(dss.CktElement.Currents())/Ibase
#     #     print(Imn)
#         Imn = Imn[0:int(len(Imn)/2)]
#     #     print(Imn)
#         Imn = Imn[0:(len(ph)*2)-1:2] + 1j*Imn[1:len(ph)*2:2]
#     #     print(Imn)
#         IDSS[ph,k1] = Imn
#     #     print('')

#     print('IDSS\n', np.round(IDSS,decimals=6))

#     STXDSS = np.zeros((3,len(dss.Lines.AllNames())),dtype='complex')
#     SRXDSS = np.zeros((3,len(dss.Lines.AllNames())),dtype='complex')

#     for k1 in range(len(dss.Lines.AllNames())):
#         dss.Lines.Name(dss.Lines.AllNames()[k1])
#         ph = np.asarray(dss.CktElement.BusNames()[0].split('.')[1:], dtype='int')-1
#         Sk = np.asarray(dss.CktElement.Powers())/(Sbase/1000)

#         STXtemp = Sk[0:int(len(Sk)/2)]
#         SRXtemp = Sk[int(len(Sk)/2):]

#         STXtemp = STXtemp[0:len(ph)*2-1:2] + 1j*STXtemp[1:len(ph)*2:2]
#         SRXtemp = -(SRXtemp[0:len(ph)*2-1:2] + 1j*SRXtemp[1:len(ph)*2:2])

#         STXDSS[ph,k1] = STXtemp
#         SRXDSS[ph,k1] = SRXtemp

    #VDSS0[i, :, :] = VDSS
#     IDSS0[i, :, :] = IDSS
#     SRXDSS0[i, :, :] = SRXDSS
#     STXDSS0[i, :, :] = STXDSS


#     print('STXDSS\n', np.round(STXDSS,decimals=6))
#     print('SRXDSS\n', np.round(SRXDSS,decimals=6))

#     print('|VDSS|\n', np.round(np.abs(VDSS),decimals=6))
#     print('<VDSS\n', np.round(180/np.pi*np.angle(VDSS),decimals=6))
#     print('D<VDSS\n', 180/np.pi*np.angle(VDSS) - 180/np.pi*np.angle(VDSS[:,[0]]))


Initial Model Converged. Proceeding to Next Step.
OpenDSS Model Compliation Done.
Iterations:  8
Tolerance:  1e-12

True
7
['sourcebus', 'a01', 'a02', 'a03', 'a04', 'a05', 'a06']
9
['load_a02_a_01', 'load_a02_b_01', 'load_a02_c_01', 'load_a03_abc_01', 'load_a04_a_01', 'load_a04_c_01', 'load_a05_ab_01', 'load_a05_a_01', 'load_a06_b_01']

VDSS
 [[ 0.999997-6.00000e-06j  0.      +0.00000e+00j  0.      +0.00000e+00j
   0.      +0.00000e+00j  0.      +0.00000e+00j  0.      +0.00000e+00j
   0.      +0.00000e+00j]
 [-0.500003-8.66021e-01j  0.      +0.00000e+00j  0.      +0.00000e+00j
   0.      +0.00000e+00j  0.      +0.00000e+00j  0.      +0.00000e+00j
   0.      +0.00000e+00j]
 [-0.499995+8.66026e-01j  0.      +0.00000e+00j  0.      +0.00000e+00j
   0.      +0.00000e+00j  0.      +0.00000e+00j  0.      +0.00000e+00j
   0.      +0.00000e+00j]]
VDSS
 [[ 0.999997-6.00000e-06j  0.998575+2.69200e-03j  0.      +0.00000e+00j
   0.      +0.00000e+00j  0.      +0.00000e+00j  0.      +0.00000e+00j
  

In [4]:
VDSS0[0]

array([[ 0.99999696-6.20229663e-06j,  0.99857518+2.69198459e-03j,
         0.99502139+9.43767880e-03j,  0.98826528+1.86939022e-02j,
         0.99136653+1.88484278e-02j,  0.97908643+2.69668895e-02j,
         0.9913667 +1.88481799e-02j],
       [-0.50000304-8.66020991e-01j, -0.49829417-8.68238643e-01j,
        -0.49402212-8.73783467e-01j, -0.48932584-8.83068865e-01j,
         0.        +0.00000000e+00j, -0.49122   -8.86192202e-01j,
        -0.49005813-8.83930183e-01j],
       [-0.49999538+8.66026069e-01j, -0.50294104+8.63285220e-01j,
        -0.51030573+8.56433567e-01j, -0.52157516+8.45198490e-01j,
        -0.52152645+8.38549183e-01j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j]])

In [5]:
VNR01[0]

array([[ 1.00009271+0.j        ,  1.00012493+0.j        ,
         1.00015826+0.j        ,  1.01039594-0.00285547j,
         1.01039594-0.00285547j,  1.01039594-0.00285547j,
         1.01039594-0.00285547j],
       [-0.44028418-0.76259456j, -0.41391989-0.71693028j,
        -0.38503513-0.66690041j, -0.33946849-0.58316038j,
         0.        +0.j        , -0.32594654-0.55995061j,
        -0.24510697-0.41877921j],
       [-0.50004635+0.86610569j, -0.50006246+0.8661336j ,
        -0.50007913+0.86616246j, -0.50272506+0.87645628j,
        -0.50272506+0.87645628j,  0.        +0.j        ,
         0.        +0.j        ]])

Comparing results between three solutions - DSS, 01 (vectorized), 02 (nonvectorized)

In [14]:
for i in range(len(times)):
    print("Timestep %d \n" % i)
    print('Complex Voltage difference')
    #print(np.max(np.abs(VNR01[i] - VNR02[i])))
    print(np.max(np.abs(VDSS0[i] - VNR01[i]))) 

#     print('Complex Current difference')
#     print(np.max(np.abs(IDSS0[i] - INR01[i])))
    
#     print('Complex TX Power difference')
#     print(np.max(np.abs(STXDSS0[i] - STXNR01[i])))
    
#     print('Complex RX Power difference')
#     print(np.max(np.abs(SRXDSS0[i] - SRXNR01[i])))
    
#     print('Voltage Magnitude difference')
#     print(np.max(np.abs(np.abs(VDSS0[i]) - np.abs(VNR01[i]))))
    
#     print('Voltage Angle difference')
#     print(np.max(np.abs(180/np.pi*np.angle(VDSS0[i]) - 180/np.pi*np.angle(VNR01[i]))))
#     print("\n")

Timestep 0 

Complex Voltage difference
0.48393996716343535


In [ ]:
np.abs(VDSS0[0]-VNR01[0])
print(np.abs(VNR01[0]))

Load comparison between non-vectorized and OpenDSS

In [ ]:
for i in range(len(times)):
    print("Timestep %d \n Magnitude of the Difference: \n" % i)
    print(np.abs(spah[i] - load_arr[i]))
    print("Max difference: \n ")
    print(np.max(np.abs(spah[i] - load_arr[i])))
    print("\n")

In [ ]:
dss.run_command('Redirect IEEE_13Node_Modified_01.dss')
for i in range(len(dss.Loads.AllNames())):
    dss.Loads.Name(dss.Loads.AllNames()[i])
    print(dss.Loads.kW())
    print(dss.Loads.kvar())
    print(dss.CktElement.Powers())

---

In [ ]:
t = -1
def load_order_f():
    load_order = {}
    for n in range(len(dss.Loads.AllNames())):
        dss.Loads.Name(dss.Loads.AllNames()[n])
        pattern =  r"(\w+)\."
        load_bus = re.findall(pattern, dss.CktElement.BusNames()[0])
        if load_bus[0] not in load_order:
            load_order[load_bus[0]] = 1
        elif load_bus[0] in load_order:
            load_order[load_bus[0]] += 1
    return load_order
load_order_list = load_order_f()

def load_values():
    load_ph_arr = np.zeros((nnode, max(load_order_list.values()), 3))
    load_kw_arr_ph = np.zeros((3, nnode))
    load_kvar_arr_ph = np.zeros((3, nnode))
    if t == -1:
        var = 1
    else:
        var = (1 + 0.1*np.sin(2*np.pi*0.01*t))
    for load in range(len(dss.Loads.AllNames())):
        dss.Loads.Name(dss.Loads.AllNames()[load])
        load_data = dss.CktElement.BusNames()[0].split('.')
        idxbs = dss.Circuit.AllBusNames().index(load_data[0])
        for ph in range(1, len(load_data)):
            load_kw_arr_ph[int(load_data[ph]) - 1, idxbs] += dss.Loads.kW() * 1e3 / Sbase / (len(load_data) - 1)
            load_kvar_arr_ph[int(load_data[ph]) - 1, idxbs] += dss.Loads.kvar() * 1e3 / Sbase / (len(load_data) - 1) 
    return load_kw_arr_ph, load_kvar_arr_ph
load_kw_arr_ph, load_kvar_arr_ph = load_values()
print(load_kw_arr_ph)
print(load_kvar_arr_ph)

In [ ]:

            idxbs = dss.Circuit.AllBusNames().index(cap_data[0])
            for ph in range(1, len(cap_data)):
                caparr[int(cap_data[ph]) - 1, idxbs] += dss.Capacitors.kvar() * 1e3 / Sbase / (len(cap_data) - 1)
        return caparr

In [ ]:
[[0.         0.         0.10384609 0.10384609 0.10384609 0.15576913
  0.        ]
 [0.         0.         0.10384609 0.10384609 0.         0.10384609
  0.05192304]
 [0.         0.         0.10384609 0.10384609 0.10384609 0.
  0.        ]]
[[0.         0.         0.02596152 0.02596152 0.02596152 0.05192304
  0.        ]
 [0.         0.         0.02596152 0.02596152 0.         0.02596152
  0.02596152]
 [0.         0.         0.02596152 0.02596152 0.02596152 0.
  0.        ]]